## 1. Datenaufbereitung (Preprocessing)

In [43]:
import pandas as pd

# Liste der Dateipfade mit dem relativen Pfad '../'
file_paths = [
    '../data/audi.csv',
    '../data/bmw.csv',
    '../data/cclass.csv',
    '../data/focus.csv',
    '../data/ford.csv',
    '../data/hyundi.csv',
    '../data/merc.csv',
    '../data/skoda.csv',
    '../data/toyota.csv',
    '../data/vauxhall.csv',
    '../data/vw.csv',
]

# Liste, um alle DataFrames zu speichern
all_dataframes = []

# Durchlaufe alle Dateipfade und lade die CSV-Dateien
for file_path in file_paths:
    df = pd.read_csv(file_path)
    brand = file_path.split('/')[-1].split('.')[0]  # Extrahiere die Marke aus dem Dateinamen
    df['brand'] = brand  # Füge eine Spalte mit der Marke hinzu
    # Ändere die Reihenfolge der Spalten, um "brand" an den Anfang zu setzen
    df = df[['brand'] + [col for col in df.columns if col != 'brand']]
    all_dataframes.append(df)

# Verbinde alle DataFrames
df = pd.concat(all_dataframes, ignore_index=True)

In [44]:
df.head()

,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,audi,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN
1,audi,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN
2,audi,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN
3,audi,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN
4,audi,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN


### Schritt 1: Überprüfung auf Duplikate

In [45]:
df.shape

(108540, 11)

In [46]:
# Überprüfung auf Duplikate
duplicate_count = df.duplicated().sum()
print(duplicate_count)

2273


In [47]:
# Entfernen von Duplikaten, falls vorhanden
if duplicate_count > 0:
    data = df.drop_duplicates()

### Schritt 2: Prüfung auf fehlende Werte

In [48]:
# Überprüfung auf fehlende Werte in jeder Spalte
missing_values = data.isnull().sum()
missing_values[missing_values > 0]

tax        13329
mpg         8555
tax(£)    101493
dtype: int64

Es gibt fehlende Werte in den Spalten tax, mpg, und tax(£). Die Spalte tax(£) scheint in einem großen Teil des Datensatzes zu fehlen. Wir könnten diese Spalte entfernen, da sie nicht viele Informationen enthält. Bei den anderen Spalten könnten wir die fehlenden Werte mit geeigneten Methoden behandeln, wie z.B. durch das Ausfüllen mit dem Durchschnittswert oder Median.

Entfernen der Spalte tax(£) und Behandeln von fehlenden Werten

In [49]:
# Entfernen der Spalte "tax(£)"
df = data.drop(columns=['tax(£)'])

# Fehlende Werte in den Spalten "tax" und "mpg" mit dem Median auffüllen
df['tax'].fillna(df['tax'].median(), inplace=True)
df['mpg'].fillna(df['mpg'].median(), inplace=True)

# Überprüfung, ob noch fehlende Werte vorhanden sind
df.isnull().sum()

brand           0
model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [50]:
print(df['year'].min())
print(df['year'].max())

1970
2060


In [51]:
# Entfernen der Zeilen, in denen das Jahr 1970 oder 2060 ist
df = df[(df['year'] != 1970) & (df['year'] != 2060)]

# Überprüfung der neuen Form des Datensatzes
df.shape

(106264, 10)

In [52]:
df['transmission'].unique()

array(['Manual', 'Automatic', 'Semi-Auto', 'Other'], dtype=object)

In [53]:
df

,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,audi,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4
1,audi,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0
2,audi,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4
3,audi,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0
4,audi,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0
...,...,...,...,...,...,...,...,...,...,...
108535,vw,Eos,2012,5990,Manual,74000,Diesel,125.0,58.9,2.0
108536,vw,Fox,2008,1799,Manual,88102,Petrol,145.0,46.3,1.2
108537,vw,Fox,2009,1590,Manual,70000,Petrol,200.0,42.0,1.4
108538,vw,Fox,2006,1250,Manual,82704,Petrol,150.0,46.3,1.2


In [54]:
# Function to convert miles per gallon (mpg) to liters per 100 kilometers (l/100km)
def mpg_to_l_per_100km(mpg):
    return 235.215 / mpg

# Function to convert miles to kilometers
def miles_to_km(miles):
    return miles * 1.60934

# Translating column names
df.columns = [
    'Marke', 'Modell', 'Baujahr', 'Preis (£)', 'Getriebe', 'Kilometerstand', 'Kraftstoffart', 'Steuer (£)', 'Verbrauch (l/100km)', 'Motorgröße (l)']

# Translating values in the 'Getriebe' column
df['Getriebe'] = df['Getriebe'].map({'Manual': 'Manuell', 
                                     'Automatic': 'Automatik', 
                                     'Semi-Auto': 'Halbautomatisch',
                                     'Other': 'Andere'})

# Translating values in the 'Kraftstoffart' column
df['Kraftstoffart'] = df['Kraftstoffart'].map({'Petrol': 'Benzin', 
                                               'Diesel': 'Diesel', 
                                               'Hybrid': 'Hybrid',
                                               'Other': 'Andere',
                                               'Electric': 'Elektrisch'})

# Converting mileage from miles to kilometers
df['Kilometerstand'] = df['Kilometerstand'].apply(miles_to_km).astype(int)

# Converting mpg to liters per 100 kilometers
df['Verbrauch (l/100km)'] = df['Verbrauch (l/100km)'].apply(mpg_to_l_per_100km).round(2)

# Capitalizing the first letter of each entry in the "Marke" column
df['Marke'] = df['Marke'].str.capitalize()


# Displaying the first few rows after rounding
df.head()


,Marke,Modell,Baujahr,Preis (£),Getriebe,Kilometerstand,Kraftstoffart,Steuer (£),Verbrauch (l/100km),Motorgröße (l)
0,Audi,A1,2017,12500,Manuell,25322,Benzin,150.0,4.25,1.4
1,Audi,A6,2016,16500,Automatik,58262,Diesel,20.0,3.66,2.0
2,Audi,A1,2016,11000,Manuell,48193,Benzin,30.0,4.25,1.4
3,Audi,A4,2017,16800,Automatik,41765,Diesel,145.0,3.50,2.0
4,Audi,A3,2019,17300,Manuell,3215,Benzin,145.0,4.74,1.0


In [55]:
df.shape

(106264, 10)

In [56]:
df.to_csv('../data (clean)/df_clean.csv', index=False)